In [ ]:
import moods
import util
import os
import numpy as np
import pandas as pd
import pomegranate
import scipy.special
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import vdom.helpers as vdomh
import tqdm
tqdm.tqdm_notebook()

In [ ]:
# Plotting defaults
plot_params = {
    "figure.titlesize": 22,
    "axes.titlesize": 22,
    "axes.labelsize": 20,
    "legend.fontsize": 18,
    "xtick.labelsize": 16,
    "ytick.labelsize": 16,
    "font.weight": "bold"
}
plt.rcParams.update(plot_params)

### Define constants and paths

In [ ]:
# moods_dir = "/users/amtseng/tfmodisco/results/moods/multitask_profile_finetune/E2F6_multitask_profile_finetune_fold1/E2F6_multitask_profile_finetune_task1_fold1_count_motif_hits"

In [ ]:
# expids = {
#     0 : "ENCSR000BGZ",
#     1 : "ENCSR000BKM",
#     2 : "ENCSR000BSE",
#     3 : "ENCSR000DTO",
#     4 : "ENCSR000EFS",
#     5 : "ENCSR000EWG",
#     6 : "ENCSR000FAH",
#     7 : "ENCSR240PRQ",
#     8 : "ENCSR725VFL"
# }
# expid = expids[5]
# key = "counts"
# moods_dir = os.path.join("/mnt/lab_data2/amtseng/tf_atlas/results/moods", expid, key)

In [ ]:
moods_dir = os.environ["TFM_MOODS_DIR"]
print("MOODS directory: %s" % moods_dir)

In [ ]:
# Constants
input_length = 2114

### Import hits and scores

In [ ]:
# Import MOODS table
hits_path = os.path.join(moods_dir, "moods_filtered_collapsed_scored.bed")
hit_table = moods.import_moods_hits(hits_path)

In [ ]:
scores = hit_table["imp_frac_score"].values
scores_finite = scores[np.isfinite(scores)]

In [ ]:
def estimate_mode(x_values, bins=200, levels=1):
    """
    Estimates the mode of the distribution using `levels`
    iterations of histograms.
    """
    hist, edges = np.histogram(x_values, bins=bins)
    bin_mode = np.argmax(hist)
    left_edge, right_edge = edges[bin_mode], edges[bin_mode + 1]
    if levels <= 1:
        return (left_edge + right_edge) / 2
    else:
        return estimate_mode(
            x_values[(x_values >= left_edge) & (x_values < right_edge)],
            bins=bins,
            levels=(levels - 1)
        )

In [ ]:
def fit_tight_exponential_dist(x_values, mode=0, percentiles=np.arange(0.05, 1, 0.05)):
    """
    Given an array of x-values and a set of percentiles of the distribution,
    computes the set of lambda values for an exponential distribution if the
    distribution were fit to each percentile of the x-values. Returns an array
    of lambda values parallel to `percentiles`. The exponential distribution
    is assumed to have the given mean/mode, and all data less than this mode
    is tossed out when doing this computation.
    """
    assert np.min(percentiles) >= 0 and np.max(percentiles) <= 1
    x_values = x_values[x_values >= mode]
    per_x_vals = np.percentile(x_values, percentiles * 100)
    return -np.log(1 - percentiles) / (per_x_vals - mode)

In [ ]:
def exponential_pdf(x_values, lamb):
    return lamb * np.exp(-lamb * x_values)
def exponential_cdf(x_values, lamb):
    return 1 - np.exp(-lamb * x_values)

In [ ]:
mode = estimate_mode(scores_finite)

In [ ]:
# Fit mixture of models to scores (mode-shifted)
over_mode_scores = scores_finite[scores_finite >= mode] - mode
mixed_model = pomegranate.GeneralMixtureModel.from_samples(
    [
        pomegranate.ExponentialDistribution,
        pomegranate.NormalDistribution,
        pomegranate.NormalDistribution
    ],
    3, over_mode_scores[:, None]
)
mixed_model = mixed_model.fit(over_mode_scores)
mixed_model_exp_dist = mixed_model.distributions[0]

In [ ]:
# Obtain a distribution of scores that belong to the exponential distribution
exp_scores = over_mode_scores[mixed_model.predict(over_mode_scores[:, None]) == 0]

In [ ]:
# Fit a tight exponential distribution based on percentiles
lamb = np.max(fit_tight_exponential_dist(exp_scores))

In [ ]:
fig, ax = plt.subplots(nrows=3, figsize=(20, 20))

x = np.linspace(np.min(scores_finite), np.max(scores_finite), 200)[1:]  # Skip first bucket (it's usually very large
mix_dist_pdf = mixed_model.probability(x)
mixed_model_exp_dist_pdf = mixed_model_exp_dist.probability(x)

perc_dist_pdf = exponential_pdf(x, lamb)
perc_dist_cdf = exponential_cdf(x, lamb)

# Plot mixed model
ax[0].hist(over_mode_scores + mode, bins=500, density=True, alpha=0.3)
ax[0].axvline(mode)
ax[0].plot(x + mode, mix_dist_pdf, label="Mixed model")
ax[0].plot(x + mode, mixed_model_exp_dist_pdf, label="Exponential component")
ax[0].legend()

# Plot fitted PDF
ax[1].hist(exp_scores, bins=500, density=True, alpha=0.3)
ax[1].plot(x + mode, perc_dist_pdf, label="Percentile-fitted")

# Plot fitted CDF
ax[2].hist(exp_scores, bins=500, density=True, alpha=1, cumulative=True, histtype="step")
ax[2].plot(x + mode, perc_dist_cdf, label="Percentile-fitted")

ax[0].set_title("Motif hit scores")
plt.show()

In [ ]:
score_range = np.linspace(np.min(scores_finite), np.max(scores_finite), 1000000)
inverse_cdf = 1 - exponential_cdf(score_range, lamb)

In [ ]:
assignments = np.digitize(scores - mode, score_range, right=True)
assignments[~np.isfinite(scores)] = 0  # If score was NaN, give it a p-value of ~1

In [ ]:
pvals = inverse_cdf[assignments]
pvals_sorted = np.sort(pvals)

In [ ]:
fdr_levels = [0.05, 0.1, 0.2, 0.3]
pval_threshes = []

fig, ax = plt.subplots(figsize=(20, 8))
ranks = np.arange(1, len(pvals_sorted) + 1)
ax.plot(ranks, pvals_sorted, color="black", label="p-values")
for fdr in fdr_levels:
    bh_crit_vals = ranks / len(ranks) * fdr
    ax.plot(ranks, bh_crit_vals, label=("Crit values (FDR = %.2f)" % fdr))
    pval_threshes.append(pvals_sorted[np.max(np.where(pvals_sorted <= bh_crit_vals)[0])])
ax.set_title("Step-up p-values and FDR corrective critical values")
plt.legend()
plt.show()

In [ ]:
# Number of hits at each FDR level
header = vdomh.thead(
    vdomh.tr(
        vdomh.th("FDR level", style={"text-align": "center"}),
        vdomh.th("Number of hits kept", style={"text-align": "center"}),
        vdomh.th("% hits kept", style={"text-align": "center"})
    )
)
rows = []
for i, fdr in enumerate(fdr_levels):
    num_kept = np.sum(pvals <= pval_threshes[i])
    frac_kept = num_kept / len(pvals)
    rows.append(vdomh.tr(
        vdomh.td("%.2f" % fdr), vdomh.td("%d" % num_kept), vdomh.td("%.2f%%" % (frac_kept * 100))
    ))
body = vdomh.tbody(*rows)
vdomh.table(header, body)